In [1]:
import psycopg2
import datetime, time
import pandas as pd
import datetime as dt
#import holoviews as hv
#hv.extension('bokeh','matplotlib')

from numpy import *
import json
#from holoviews import opts
#from bokeh.models import DatetimeTickFormatter, Legend

#from monitor import *

In [2]:
properties = { 'cpuload' : {'table_name' :'containerstats_cpuload',
                            'unit' : '%',
                            'conv' : 1/100,
                            'resample': 'sum',
                           },
                'memoryusage' : {'table_name' :'containerstats_mem',
                            'unit' : 'MB',
                            'conv' : 1/1024**2,
                            'resample': 'max',
                           },  
                'pids' : {'table_name' :'containerstats_pids',
                            'unit' : '',
                            'conv' : 1,
                            'resample': 'max',
                           },  
                'net_i' : {'table_name' :'containerstats_net_i',
                            'unit' : 'MB',
                            'conv' : 1/1024**2,
                            'resample': 'sum',
                           },
                'net_o' : {'table_name' :'containerstats_net_o',
                            'unit' : 'MB',
                            'conv' : 1/1024**2,
                            'resample': 'sum',
                           },
                'block_i' : {'table_name' :'containerstats_block_i',
                            'unit' : 'MB',
                            'conv' : 1/1024**2,
                             'resample': 'sum',
                           },
                'block_o' : {'table_name' :'containerstats_block_o',
                            'unit' : 'MB',
                            'conv' : 1/1024**2,
                            'resample': 'sum',
                           }
                 }


In [3]:
REQUEST = json.dumps({
'container' : {},
'user' : {},
'project' : {},
'last' : {},
'stat' : {},
'args' : {}
})

In [4]:
# GET /stats/user
req = json.loads(REQUEST)
args = req['args']

P = properties

total_cpu=0
current_mem = 0 
current_cpu = 0
cpus={}
mems={}
if 'user_name' not in args:
    print(json.dumps({'Error': 'No user_name was supplied'}))
else: # 'user_name' in args:
    # Note the [0] when retrieving the argument.
    user_name = args['user_name'][0]
    Usage = Usage_Data()
    cont_list = Usage.select_user_containers(user_name)
    Usage.gather_data(cont_list)
    

    for cid in Usage.selected_containers:
        cont_data = Usage.containers[cid]
        current_cpu += cont_data.last_data('cpuload')
        cpus[cont_data.name] = cont_data.last_data('cpuload')
        
        current_mem += cont_data.last_data('memoryusage')*P['memoryusage']['conv']
        mems[cont_data.name] = cont_data.last_data('memoryusage')*P['memoryusage']['conv']

    scurrent_cpu = "%.2f%s CPU"%(float(current_cpu), P['cpuload']['unit'])
    scurrent_mem = "%d %s"%(int(current_mem),  P['memoryusage']['unit'])
    stotal_cpu = "%.2f%s CPU"%(float(total_cpu), P['cpuload']['unit'])
    
    print(json.dumps({'current_cpu': scurrent_cpu, 'current_mem': scurrent_mem,
                      'total_cpu': stotal_cpu, 'cpus': cpus,
                     'mems': mems}))



{"Error": "No user_name was supplied"}


In [5]:
# GET /stats/project
req = json.loads(REQUEST)
args = req['args']

P = properties

total_cpu=0
current_mem = 0 
current_cpu = 0
cpus={}
mems={}
if 'project_name' not in args:
    print(json.dumps({'Error': 'No project_name was supplied'}))
else: # 'user_name' in args:
    # Note the [0] when retrieving the argument.
    project_name = args['project_name'][0]
    Usage = Usage_Data()
    cont_list = Usage.select_project_containers(project_name)
    Usage.gather_data(cont_list)
    
    for cid in Usage.selected_containers:
        cont_data = Usage.containers[cid]
        current_cpu += cont_data.last_data('cpuload')
        cpus[cont_data.name] = cont_data.last_data('cpuload')
        
        current_mem += cont_data.last_data('memoryusage')*P['memoryusage']['conv']
        mems[cont_data.name] = cont_data.last_data('memoryusage')*P['memoryusage']['conv']

    scurrent_cpu = "%.2f%s CPU"%(float(current_cpu), P['cpuload']['unit'])
    scurrent_mem = "%d %s"%(int(current_mem),  P['memoryusage']['unit'])
    stotal_cpu = "%.2f%s CPU"%(float(total_cpu), P['cpuload']['unit'])
    
    print(json.dumps({'current_cpu': scurrent_cpu, 'current_mem': scurrent_mem,
                      'total_cpu': stotal_cpu, 'cpus': cpus,
                     'mems': mems}))

{"Error": "No project_name was supplied"}


In [7]:
# GET /stats/stats
req = json.loads(REQUEST)
args = req['args']

P = properties

if 'all_containers' in args:
    Usage = Usage_Data()
    print(Usage.number_of_containers)
elif 'running_containers' in args:
    Usage = Usage_Data()
    len(self.containers_info[self.containers_info.running==True])
elif 'current_load' in args:
    Usage = Usage_Data()
    Usage.gather_data()
    total_cpu=0
    current_mem = 0 
    current_cpu = 0
    for cid in Usage.running_containers:
        cont_data = Usage.containers[cid]
        current_cpu += cont_data.last_data('cpuload')     
        current_mem += cont_data.last_data('memoryusage')*P['memoryusage']['conv']

    scurrent_cpu = "%.2f%s CPU"%(float(current_cpu), P['cpuload']['unit'])
    scurrent_mem = "%d %s"%(int(current_mem),  P['memoryusage']['unit'])
    stotal_cpu = "0"
    
    print(json.dumps({'current_cpu': scurrent_cpu, 'current_mem': scurrent_mem,
                      'total_cpu': stotal_cpu}))
else:
    print(json.dumps({'GET':['current_load','running_containers','all_containers']}))
    

{"GET": ["current_load", "running_containers", "all_containers"]}


In [ ]:
# GET /stats/help
req = json.loads(REQUEST)
args = req['args']

print(json.dumps({'GET':{'stats': {'current_load':['current_cpu', 'current_mem', 'total_cpu'],
                                   'running_containers': 'int', 'all_containers': 'int'},
                         'user': {'user_name': ['current_cpu','current_mem', 'total_cpu', 'cpus', 'mems']},
                         'project': {'project_name': ['current_cpu','current_mem', 'total_cpu', 'cpus', 'mems']},
                        }
                 }
                )
     )

In [184]:
# Usage = Usage_Data()
# Usage.gather_data(last=True)
# total_cpu=0
# current_mem = 0 
# current_cpu = 0

# for cid in Usage.running_containers:
#     cont_data = Usage.containers[cid]
#     current_cpu += cont_data.last_data('cpuload')     
#     current_mem += cont_data.last_data('memoryusage')*P['memoryusage']['conv']

# scurrent_cpu = "%.2f%s CPU"%(float(current_cpu), P['cpuload']['unit'])
# scurrent_mem = "%d %s"%(int(current_mem),  P['memoryusage']['unit'])

# print(json.dumps({'current_cpu': scurrent_cpu, 'current_mem': scurrent_mem,
#                   'total_cpu': stotal_cpu}))

{"current_cpu": "2103.00% CPU", "current_mem": "104209 MB", "total_cpu": "0.00% CPU"}


In [9]:
def connect_to_database():
    conn = psycopg2.connect("host=kooplex-monitordb dbname=monitor user=usage_viewer password=whatisusage")
    return conn
    
class Container():
    """
    A docker container that belongs to a certain user and project
    """
    
    def __init__(self, c_id):
        """
        :param tables: dict of queried sql tables
        :param cid: docker container id
        :param name: docker container name
        :param running: whether docker container is running now
        :param last: collect only the last some data
        """
        
        self.tables = {}
        self.cid = c_id
        self.name = ""
        self.running = False
        self.user_name = ""
        self.project_name = ""
        self.last = False
        #self._get_name()

    def get_usage(self, prop):
        """
        Get 'cpuload', 'memoryusage' etc. for a certain user and project
        """
        
        table_name = properties[prop]['table_name']
        if self.last:
            sqlc = "select * from %s where container_id = %d ORDER BY ID DESC LIMIT 1" % (table_name, self.cid)            
        else:
            sqlc = "select * from %s where container_id = %d" % (table_name, self.cid)
         
        tdf = pd.read_sql(sqlc, connect_to_database())                                      
        tdf['datetime'] = pd.to_datetime(tdf['last_read'], unit='s')
        tdf = tdf.set_index('datetime')            
        tdf.drop(['last_read'], axis=1, inplace=True)
        
        if tdf.empty:
            #print("%s EMPTY ERROR"%prop)
            self.tables[prop] = tdf
            return
        
        tdf = tdf.astype(float)
        
        if not self.last:
            if prop == 'cpuload':
                tdf = tdf.resample('2T').mean()
            elif prop == 'memoryusage':
                tdf = tdf.resample('2T').ffill()

            tdf = tdf.fillna(value=0)
            
        self.tables[prop] = tdf

    def last_data(self, prop):
        tdf = self.tables[prop]
        tt = tdf[(datetime.datetime.now()-datetime.timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M"):]
        if not self.running:
            return 0
        if prop == 'cpuload':
            if len(tt) == 0:
                return 0
            else:
                tt = tt.sort_index()
                return tt.iloc[-1][prop]
        else:
            return tdf.iloc[-1][prop]
    

    
class Usage_Data():
    def __init__(self):
        self.usage_dict = {}
        self.usage = {}
        self.containers = {}
        self.containers_info = pd.DataFrame(columns=['id', 'container_name', 'project', 'username'])
        self.selected_containers = []
        self.get_all_containers()
        self.total_memory = 0
        self.total_cpu = 0     
        #self.temp_df = None
        self.prop_to_use = None
        
        
    def get_all_containers(self):
        sqlc = "select ff.id, project, username, container_name, running from (select conts.id, project, hubuser, \
        container_name, running from container as conts inner join project as proj on conts.project_id = proj.id) \
        as ff inner join hubuser on ff.hubuser = hubuser.id;"
        self.containers_info = pd.read_sql(sqlc, connect_to_database())
        self.selected_containers = list(self.containers_info.id)        
        
    @property
    def number_of_containers(self):
        return  len(self.containers_info)

    @property
    def running_containers(self):
        return self.containers_info[self.containers_info.running==True].id
    
    def select_project_containers(self, project_name):
        c_ids = self.containers_info[self.containers_info.project == project_name].id
        self.selected_containers = list(c_ids)
        return self.selected_containers
        
    def select_user_containers(self, user_name):
        c_ids = self.containers_info[self.containers_info.username == user_name].id
        self.selected_containers = list(c_ids)
        return self.selected_containers        
    
    @property
    def list_all_projects(self):
        return Usage.containers_info.drop_duplicates()

    @property
    def list_all_users(self):
        return Usage.containers_info.username.drop_duplicates()

    @property
    def list_all_containers(self):
        return Usage.containers_info
    
    def gather_data(self, container_ids=[], properties=[], last=False):
        self.containers = {}
        self.get_all_containers()

        if container_ids:
            self.selected_containers  = container_ids
        
        self.selected_containers = intersect1d(self.selected_containers, self.running_containers)
        for i in self.selected_containers:
            C = Container(i)
                        
            C.last = last
            #C.get_cpu_usage()
            #C.get_memory_usage()
            C.get_usage('cpuload')
            C.get_usage('memoryusage')
            info_for_C = self.containers_info[self.containers_info.id==C.cid]
            C.running = info_for_C['running'].iloc[0]
            C.user_name = info_for_C['username'].iloc[0]
            C.name = info_for_C['container_name'].iloc[0]
            self.containers[C.cid] = C

    def monthly_usage(self, prop, trunc):
        if prop==1:
            self.prop_to_use = 'cpuload'
        elif prop==2:
            self.prop_to_use = 'memoryusage'       
        return self._usage( sample_rate='m', trunc=trunc*30)
    
    #def weekly_usage(self, prop='cpuload', trunc=1):
    #def weekly_usage(self, prop=1, trunc=1):
    def weekly_usage(self, prop, trunc):
        if prop==1:
            self.prop_to_use = 'cpuload'
        elif prop==2:
            self.prop_to_use = 'memoryusage'
        return self._usage(sample_rate='w', trunc=trunc*7)

    #def daily_usage(self, prop='cpuload', trunc=1):
    def daily_usage(self, prop, trunc):
        if prop==1:
            self.prop_to_use = 'cpuload'
        elif prop==2:
            self.prop_to_use = 'memoryusage'
        return self._usage(sample_rate='d', trunc=trunc)
    
    def daily_usage_s(self, prop='cpuload', trunc=30):
        self.prop_to_use = prop
        return self._usage(sample_rate='d', trunc=trunc)
    
    def myplot(self):
        bars = hv.Bars(self.temp_df, ['datetime', 'project_name'], self.prop_to_use)
        return bars.relabel(group='Stacked')
    
    def _usage(self, sample_rate='d', trunc=1):
        #temporary df
        self.temp_df = pd.DataFrame()
        for cid in self.selected_containers:
            # select container data
            C = self.containers[cid]['container']
            # get the relevant table
            C_t = C.tables[self.prop_to_use].copy()
            # set the relevant time interval
            start_time = dt.datetime.now() - dt.timedelta(trunc) 
            # sort indices (it might be reordered)
            C_t.df = C_t.df.sort_index()
            # truncate data according t the relevant time interval
            C_t.df = C_t.df.truncate(before=start_time)
            # do anything only if we still have data
            if not C_t.is_empty:
                C_t.resample_df(sample_rate)
                C_t.add_labels()
                # put pd.Series into a pd.DataFrame
                self.temp_df = pd.concat([ self.temp_df, C_t.df])
                
        #return self.myplot()
        
   

In [13]:
user_name='jegesm'
Usage = Usage_Data()
cont_list = Usage.select_user_containers(user_name)
Usage.gather_data(cont_list, properties=[], last=True)

P = properties
mems = {}
cpus = {}
current_cpu = 0
total_cpu = 0
current_mem = 0
for cid in Usage.selected_containers:
    cont_data = Usage.containers[cid]
    current_cpu += cont_data.last_data('cpuload')
    cpus[cont_data.name] = cont_data.last_data('cpuload')

    current_mem += cont_data.last_data('memoryusage')*P['memoryusage']['conv']
    mems[cont_data.name] = cont_data.last_data('memoryusage')*P['memoryusage']['conv']

scurrent_cpu = "%.2f%s CPU"%(float(current_cpu), P['cpuload']['unit'])
scurrent_mem = "%d %s"%(int(current_mem),  P['memoryusage']['unit'])
stotal_cpu = "%.2f%s CPU"%(float(total_cpu), P['cpuload']['unit'])

print(json.dumps({'current_cpu': scurrent_cpu, 'current_mem': scurrent_mem,
                  'total_cpu': stotal_cpu, 'cpus': cpus,
                 'mems': mems}))

{"current_cpu": "84.00% CPU", "current_mem": "2196 MB", "total_cpu": "0.00% CPU", "cpus": {"/nb-jegesm-testall-jegesm": 84.0, "/nb-jegesm-integrin-jegesm": 0, "/nb-jegesm-test-jegesm": 0, "/nb-jegesm-adatokgreen-jegesm": 0, "/nb-jegesm-dataexplorationandvisualisation-jegesm": 0, "/nb-jegesm-tesztelni-jegesm": 0}, "mems": {"/nb-jegesm-testall-jegesm": 1706.09765625, "/nb-jegesm-integrin-jegesm": 58.5078125, "/nb-jegesm-test-jegesm": 206.203125, "/nb-jegesm-adatokgreen-jegesm": 60.91796875, "/nb-jegesm-dataexplorationandvisualisation-jegesm": 57.640625, "/nb-jegesm-tesztelni-jegesm": 107.5625}}


In [151]:
# project_name='exosome'
# Usage = Usage_Data()
# cont_list = Usage.select_project_containers(project_name)
# Usage.gather_data(cont_list, properties=[], last=True)

# P = properties
# mems = {}
# cpus = {}
# current_cpu = 0
# total_cpu = 0
# current_mem = 0
# for cid in cont_list:
#     cont_data = Usage.containers[cid]
#     current_cpu += cont_data.last_data('cpuload')
#     cpus[cont_data.name] = cont_data.last_data('cpuload')

#     current_mem += cont_data.last_data('memoryusage')*P['memoryusage']['conv']
#     mems[cont_data.name] = cont_data.last_data('memoryusage')*P['memoryusage']['conv']

# scurrent_cpu = "%.2f%s CPU"%(float(current_cpu), P['cpuload']['unit'])
# scurrent_mem = "%d %s"%(int(current_mem),  P['memoryusage']['unit'])
# stotal_cpu = "%.2f%s CPU"%(float(total_cpu), P['cpuload']['unit'])

# print(json.dumps({'current_cpu': scurrent_cpu, 'current_mem': scurrent_mem,
#                   'total_cpu': stotal_cpu, 'cpus': cpus,
#                  'mems': mems}))


cpuload EMPTY ERROR
cpuload EMPTY ERROR
{"current_cpu": "0.00% CPU", "current_mem": "401 MB", "total_cpu": "0.00% CPU", "cpus": {"/nb-dlakatos-test-dlakatos": 0, "/nb-tradnai-test-dlakatos": 0, "/nb-zudvarnoki-test-andras.major": 0, "/nb-jborcsok-test-andras.major": 0, "/nb-icsabai-test-andras.major": 0, "/nb-ppollner-test-andras.major": 0, "/nb-jegesm-test-jegesm": 0, "/nb-andras.major-test-andras.major": 0, "/nb-opipek-test-andras.major": 0, "/nb-gpalla-test-andras.major": 0}, "mems": {"/nb-dlakatos-test-dlakatos": 0.0, "/nb-tradnai-test-dlakatos": 0.0, "/nb-zudvarnoki-test-andras.major": 0.0, "/nb-jborcsok-test-andras.major": 0.0, "/nb-icsabai-test-andras.major": 0.0, "/nb-ppollner-test-andras.major": 0.0, "/nb-jegesm-test-jegesm": 211.12109375, "/nb-andras.major-test-andras.major": 80.76171875, "/nb-opipek-test-andras.major": 57.33984375, "/nb-gpalla-test-andras.major": 52.23828125}}


In [125]:
# user_name='jegesm'
# Usage = Usage_Data()
# cont_list = Usage.select_user_containers(user_name)
# Usage.gather_data(cont_list, properties=[], last=False)
